1. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát éves bontásban norbert2 azonosítójú ügyfél rendeléseinek értékéről!</span>

1. A lista megfelelően jelölve tartalmazza a végösszeget is!

In [4]:
-- rendelés és rendelés_tétel táblák
-- érték: SUM(mennyiseg*egysegar)
SELECT IIF(YEAR(r.REND_DATUM) is null, 'Összesen',Cast(Year(r.REND_DATUM) as nvarchar(4))),
    SUM(rt.MENNYISEG)
    from Rendeles r join Rendeles_tetel rt on r.SORSZAM = rt.SORSZAM
    where r.[LOGIN]='norbert2'
    group by rollup(year(r.REND_DATUM))

(3 rows affected)

Total execution time: 00:00:00.030

(No column name),(No column name)
2015,871
2017,1150
Összesen,2021


2. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát szállítási dátumonként, azon belül szállítási módonként az egyes rendelések összmennyiségéről!</span>

1. Csak azokat a termékeket vegyük figyelembe, amelyek mennyiségi egysége db!
2. A listát szűrjük úgy, hogy az csak a részösszeg sorokat és a végösszeget tartalmazza!

In [8]:
-- rendeles, rendeles_tetel, termék táblák
-- SUM(mennyiseg)
SELECT r.SZALL_DATUM,r.SZALL_MOD, SUM(rt.MENNYISEG)
    from    Rendeles r  join Rendeles_tetel rt on r.SORSZAM = rt.SORSZAM
                        join Termek t on rt.TERMEKKOD = t.TERMEKKOD
    where t.MEGYS ='db'
    group by rollup(r.SZALL_DATUM,r.SZALL_MOD)
    HAVING r.SZALL_MOD is null or r.SZALL_DATUM is NULL

(711 rows affected)

Total execution time: 00:00:00.218

SZALL_DATUM,SZALL_MOD,(No column name)
2015-05-03,NULL,100
2015-05-04,NULL,996
2015-05-05,NULL,3152
2015-05-06,NULL,5030
2015-05-07,NULL,3828
2015-05-08,NULL,3213
2015-05-09,NULL,6665
2015-05-10,NULL,6909
2015-05-11,NULL,6224
2015-05-12,NULL,7568


```
3. Hány olyan ügyfél van, aki még nem rendelt semmit?
```

1. Csoportosítsuk őket nem szerint, azon belül életkor szerint!
2. A lista tartalmazza a részösszegeket és a végösszeget is!

![](https://moodle.uni-corvinus.hu/theme/image.php/adaptable/atto_h5p/1709548862/icon)

4. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát a megrendelt termékek legkisebb és legnagyobb egységáráról szállítási dátum, azon belül szállítási mód szerinti bontásban!</span>

1. A lista csak a 2015 májusi szállításokat tartalmazza!
2. Jelenítsük meg a részösszegeket és a végösszeget is!

5. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk csoportot a termékek listaára alapján a következők szerint:&nbsp;<br></span> <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Az "olcsó" termékek legyenek azok, amelyek listaára 3000 alatt van.<br>&nbsp;A "drága" termékek legyenek az 5000 felettiek, a többi legyen "közepes".</span>

1. Listázzuk az egyes csoportokat, és a csoportokba tartozó termékek darabszámát!
2. A lista jelenítse meg a végösszeget is!

6. <span style="font-weight: 600; color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát az ügyfelek adatairól név szerinti sorrendben.</span>

1. Minden sorban jelenjen meg a sorrend szerint előző, illetve következő ügyfél neve is.
2. Ha nincs előző vagy következő ügyfél, akkor a 'Nincs' jelenjen meg!

In [ ]:
--lag és lead fv-ek
select *,
        --előző ügyfél neve
        Lag(Nev) over(PARTITION by 
        --következő ügyfél neve
from Ugyfel

7. <span style="font-weight: 600; color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk lekérdezést, amely megmutatja, hogy melyik termékkategóriába hány termék tartozik. A lista a kategória nevét és a darabszámot jelenítse meg.</span>

1. A lista ne tartalmazzon duplikált sorokat.
2. A feladatot partíciók segítségével oldjuk meg!

In [12]:
select DISTINCT tk.KAT_NEV,
        count(*) OVER(PARTITION by tk.KAT_NEV)
from Termekkategoria tk join termek t on tk.KAT_ID = t.KAT_ID
--group by tk.KAT_NEV


(143 rows affected)

Total execution time: 00:00:00.016

KAT_NEV,(No column name)
ajándék tárgy,3
ajándék tasak,20
álarc,3
alkalmi,3
alufólia,2
árazószalag,1
átírótömb,1
beíró,1
beírókönyv,2
bélyegző,4


8. <span style="font-weight: 600; color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát a rendelési tételekről. Az egyes rendelési tételeket termékenként soroljuk be 4 osztályba a rendelés mennyisége alapján. Jelenítsük meg ezt az információt is egy új oszlopban, az oszlop neve legyen 'Mennyiségi kategória'.</span>

1. A lista csak a 100 Ft feletti egységárú rendelési tételeket vegye figyelembe!

In [16]:
--ntile()
select  *,
        NTILE(4) over(PARTITION by termekkod order by mennyiseg)
from Rendeles_tetel
where EGYSEGAR > 100

(91878 rows affected)

Displaying Top 5000 rows.

Total execution time: 00:00:00.871

SORSZAM,TERMEKKOD,EGYSEGAR,MENNYISEG,(No column name)
587,01010001T,310,10,1
1493,01010001T,310,10,1
3895,01010001T,310,10,1
5974,01010001T,310,10,1
7036,01010001T,310,10,1
7271,01010001T,310,10,1
7474,01010001T,310,10,1
9309,01010001T,310,10,1
11566,01010001T,342,10,1
13399,01010001T,342,10,1


9. <span style="font-weight: 600; color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Listázzuk a termékek kódját, megnevezését, kategóriájának nevét, és listaárát.</span>

1. A listát egészítsük ki két új oszloppal, amelyek a kategória legolcsóbb, illetve legdrágább termékének árát tartalmazzák.
2. A két új oszlop létrehozásánál partíciókkal dolgozzunk!

In [ ]:
--min/max vagy first_value/last_value

10. <span style="font-weight: 600; color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát a rendelésekről. A lista legyen rendezve ügyfelenként (LOGIN), azon belül a rendelés dátuma szerint. A listához készítsünk sorszámozást is. A sorszám a következő formában jelenjen meg: sorszám_év_login. Pl: 1_2015_adam1</span>

1. A számozás login-onként, azon belül rendelési évenként kezdődjön újra.
2. A sorszám oszlop neve legyen Azonosító

11. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát a termékek adatairól listaár szerint növekvő sorrendben! A lista jelenítse meg két új oszlopban a sorrend szerint előző, illetve következő termék listaárát is a termék saját kategóriájában és raktárában!</span>

1. Ahol nincs előző vagy következő érték, ott 0 jelenjen meg!
2. Az oszlopokat nevezzük el értelemszerűen!

12. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Listázzuk a termékek kódját, nevét és listaárát listaár szerinti sorrendben!</span>

1. Vegyünk fel egy új oszlopot Mozgóátlag néven, amely minden esetben az aktuális termék, az előző, és a következő termék átlagárát tartalmazza!
2. A mozgóátlagot kerekítsük két tizedesre!

In [13]:
select termekkod,
        megnevezes,
        listaar,
        --mozgóátlag
        avg(Listaar) over(order by listaar
                rows between 1 preceding and 1 following)
from termek

(551 rows affected)

Total execution time: 00:00:00.014

termekkod,megnevezes,listaar,(No column name)
07030108T,Filctoll CD író,0,"1,5"
01090498T,"Műnyomó 300g 17,5*20",3,2
02030042T,Boríték LC 6,3,3
04030288T,Műanyag pohár 1 dl-es,3,"3,3333333333333335"
01090501T,Kreatív A/4 300g,4,"3,6666666666666665"
01090517T,Színes ofszet B/4,4,"4,333333333333333"
06040089T,Ellenőrzőborító,5,5
02030040T,Boríték LA 4 francia,6,"5,666666666666667"
02030041T,Boríték LC 5,6,"6,666666666666667"
02040147T,Genotherm lefűzhető A/5,8,"7,333333333333333"


13. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát, amely a rendelések sorszámát és a rendelés értékét tartalmazza. A listát egészítsük ki egy új oszloppal, amely minden rendelés esetén addigi rendelések összegét tartalmazza (az aktuálisat is beleértve)!</span>

1. A listát rendezzük sorszám szerint növekvő sorrendbe.
2. A lista ne tartalmazzon duplikált sorokat!

14. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát a termékek kódjáról, nevéről, kategória azonosítójáról, raktár azonosítójáról és listaáráról, valamint a termék adott szempontok szerinti rangsorokban elfoglalt helyezéseiről. (Szempontonként külön oszlopban, a helyezéseknél növekvő sorrendet feltételezve). A szempontok a következők legyenek: listaár, kategória szerinti listaár, és raktárkód szerinti listaár.</span>

1. Az oszlopokat nevezzük el értelemszerűen.
2. A helyezések egyenlőség esetén "sűrűn" kövessék egymást.
3. A lista legyen rendezett kategória azonosító, azon belül listaár szerint!

15. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát a rendelési tételekről, amely minden sor esetén göngyölítve tartalmazza az ügyfél adott rendelési tételig meglévő rendelési tételeinek összértékét!</span>

1. Az új oszlop neve legyen Eddigi rendelési tételek összértéke!
2. Az ügyfél neve is jelenjen meg!